In [1]:
from pathlib import Path

import pandas as pd
from aiopvpc import PVPCData
from pvpcbill import (
    FacturaData, 
    FacturaElec, 
    FacturaConfig, 
    create_bill, 
    get_pvpc_data, 
    load_csv_consumo_cups,
    models,
)

## Recuperación de datos de consumo

Importar datos horarios de los ficheros CSV de descarga en la web official.

In [2]:
# Path al CSV de consumos horarios

p_ejemplos = Path(models.__file__).parents[1] / "tests" / "ejemplos_consumo"
assert p_ejemplos.exists(), "Fix the path to the CSV!!"

p_consumo = p_ejemplos / "consumo_facturado18_02_2020-18_03_2020-R.csv"
assert p_consumo.exists(), "Fix the path to the CSV!!"

# Show header of file
print(
    f"** CSV consumo: '{p_consumo.name}', con la forma:\n\n\t"
    + "\n\t".join(p_consumo.read_text().splitlines()[:5])
)

** CSV consumo: 'consumo_facturado18_02_2020-18_03_2020-R.csv', con la forma:

	CUPS;Fecha;Hora;Consumo_kWh;Metodo_obtencion
	ES0012345678901234SN;18/02/2020;1;0,350;R
	ES0012345678901234SN;18/02/2020;2;0,242;R
	ES0012345678901234SN;18/02/2020;3;0,239;R
	ES0012345678901234SN;18/02/2020;4;0,238;R


In [3]:
# Parser de CSVs de consumo con formato estándar a pd.Series con DateTimeIndex
s_consumo = load_csv_consumo_cups(p_consumo)
s_consumo

2020-02-18 00:00:00+01:00    0.350
2020-02-18 01:00:00+01:00    0.242
2020-02-18 02:00:00+01:00    0.239
2020-02-18 03:00:00+01:00    0.238
2020-02-18 04:00:00+01:00    0.236
                             ...  
2020-03-18 19:00:00+01:00    0.404
2020-03-18 20:00:00+01:00    0.398
2020-03-18 21:00:00+01:00    0.374
2020-03-18 22:00:00+01:00    0.446
2020-03-18 23:00:00+01:00    0.335
Name: ES0012345678901234SN, Length: 720, dtype: float64

## Obtención de datos de PVPC

Para el periodo de aplicación marcado por los datos de consumo horario

In [4]:
pvpc_handler = PVPCData()

pvpc_data = await pvpc_handler.async_download_prices_for_range(
    start=s_consumo.index[0].to_pydatetime(),
    end=s_consumo.index[-1].to_pydatetime(),
)

df_pvpc = pd.DataFrame(pvpc_data).T.set_index(s_consumo.index)
df_pvpc

Download of 720 prices from 2020-02-17 23:00:00+00:00 to 2020-03-18 22:00:00+00:00 in 0.38 sec


,GEN,NOC,VHC,COFGEN,COFNOC,COFVHC,PMHGEN,PMHNOC,PMHVHC,SAHGEN,...,INTVHC,PCAPGEN,PCAPNOC,PCAPVHC,TEUGEN,TEUNOC,TEUVHC,CCVGEN,CCVNOC,CCVVHC
2020-02-18 00:00:00+01:00,96.11,47.44,50.80,0.000102,0.000175,0.000153,42.16,40.31,42.40,2.22,...,0.04,5.70,0.95,1.35,44.03,2.22,2.88,1.76,1.62,1.71
2020-02-18 01:00:00+01:00,93.74,45.04,42.19,0.000080,0.000155,0.000173,39.50,37.69,36.52,2.47,...,0.03,5.76,0.95,0.74,44.03,2.22,0.89,1.74,1.59,1.54
2020-02-18 02:00:00+01:00,97.20,48.25,45.30,0.000069,0.000141,0.000171,42.94,40.91,39.64,2.41,...,0.03,5.78,0.96,0.74,44.03,2.22,0.89,1.79,1.64,1.59
2020-02-18 03:00:00+01:00,96.61,47.69,44.72,0.000065,0.000133,0.000163,41.76,39.79,38.52,3.00,...,0.03,5.79,0.96,0.74,44.03,2.22,0.89,1.79,1.63,1.58
2020-02-18 04:00:00+01:00,94.62,45.80,42.93,0.000063,0.000131,0.000155,39.66,37.79,36.62,3.15,...,0.03,5.79,0.96,0.74,44.03,2.22,0.89,1.76,1.60,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-18 19:00:00+01:00,99.78,118.37,118.37,0.000150,0.000125,0.000107,46.56,46.92,46.92,1.88,...,0.03,5.34,5.54,5.54,44.03,62.01,62.01,1.75,1.77,1.77
2020-03-18 20:00:00+01:00,99.64,118.22,118.22,0.000172,0.000160,0.000153,46.45,46.81,46.81,1.80,...,0.04,5.38,5.58,5.58,44.03,62.01,62.01,1.76,1.78,1.78
2020-03-18 21:00:00+01:00,97.21,115.82,115.82,0.000176,0.000174,0.000161,43.83,44.21,44.21,2.01,...,0.04,5.38,5.60,5.60,44.03,62.01,62.01,1.72,1.74,1.74
2020-03-18 22:00:00+01:00,95.59,47.38,114.22,0.000153,0.000185,0.000155,41.79,40.21,42.18,2.30,...,0.04,5.51,0.92,5.73,44.03,2.22,62.01,1.72,1.59,1.74


## Simulación de factura

Con la serie de datos de consumo horarios y la información del PVPC para ese intervalo, sólo resta por
especificar datos sobre el contrato de suministro, como:

* El peaje contratado (GEN / NOC / VHC, para 2.0A / 2.0DHA / 2.0DHS)
* La potencia contratada en kW
* La zona de impuestos (IVA / IGIC / IPSI)

In [5]:
factura = FacturaElec(
    consumo_horario=s_consumo,
    pvpc_data=df_pvpc,
    tipo_peaje="NOC",
    potencia_contratada=4.6,
    zona_impuestos="IVA",
)

# Reparto de costes en la factura
p_imp = (
    + factura.data.termino_impuesto_electrico
    + factura.data.termino_equipo_medida
    + factura.data.termino_iva_total
) / factura.data.total
p_ener = factura.data.termino_variable_total / factura.data.total
p_pot = factura.data.termino_fijo_total / factura.data.total
assert abs(p_pot + p_ener + p_imp - 1) < 0.001

print(
    f"El coste de la factura se reparte en:\n  "
    f"* un {100*p_ener:.1f} % por energía consumida,\n  "
    f"* un {100*p_pot:.1f} % por potencia contratada,\n  "
    f"* un {100*p_imp:.1f} % por impuestos aplicados\n\n"
)

# La representación del objeto como texto simula el desglose de la factura eléctrica
factura

El coste de la factura se reparte en:
  * un 52.6 % por energía consumida,
  * un 24.8 % por potencia contratada,
  * un 22.6 % por impuestos aplicados




FACTURA ELÉCTRICA:
--------------------------------------------------------------------------------
* CUPS             	        ES00XXXXXXXXXXXXXXDB
* Fecha inicio             	17/02/2020
* Fecha final              	18/03/2020
* Peaje de acceso          	2.0DHA (Nocturna)
* Potencia contratada      	4.60 kW
* Consumo periodo          	472.93 kWh
* ¿Bono Social?            	No
* Equipo de medida         	0.80 €
* Impuestos                	Península y Baleares (IVA)
* Días facturables         	30
--------------------------------------------------------------------------------

- CÁLCULO DEL TÉRMINO FIJO POR POTENCIA CONTRATADA:
  Peaje acceso potencia:
   4.60 kW x 0.103944 €/kW/día x 30 días (366/2020) = 14.34 €
  Comercialización:
   4.60 kW x 0.008505 €/kW/día x 30 días (366/2020) = 1.17 €
  ==> Término fijo                                                     15.51 €

- CÁLCULO DEL TÉRMINO VARIABLE POR ENERGÍA CONSUMIDA (TARIFA 2.0DHA):
    Periodo 1: 0.111867 €/kWh                   

In [6]:
# pero también existen otras formas de acceder a la información contenida:
factura.data.to_dict()

{'config': {'tipo_peaje': 'NOC',
  'potencia_contratada': 4.6,
  'con_bono_social': False,
  'zona_impuestos': 'IVA',
  'alquiler_anual': 9.72,
  'impuesto_electrico': 0.0511269632,
  'cups': 'ES00XXXXXXXXXXXXXXDB'},
 'num_dias_factura': 30,
 'start': '2020-02-17 00:00:00',
 'end': '2020-03-18 00:00:00',
 'periodos_fact': [{'billed_days': 30,
   'year': 2020,
   'termino_fijo_peaje_acceso': 14.34,
   'termino_fijo_comercializacion': 1.17,
   'termino_fijo_total': 15.51,
   'energy_periods': [{'name': 'P1',
     'coste_peaje_acceso_tea': 10.544458468,
     'coste_energia_tcu': 8.477372039999999,
     'energia_total': 170.03900000000002},
    {'name': 'P2',
     'coste_peaje_acceso_tea': 0.67090578,
     'coste_energia_tcu': 13.146024950000003,
     'energia_total': 302.892}]}],
 'descuento_bono_social': 0.0,
 'termino_impuesto_electrico': 2.47,
 'termino_equipo_medida': 0.8,
 'termino_iva_gen': 10.6722,
 'termino_iva_medida': 0.168,
 'termino_iva_total': 10.84,
 'total': 62.46}

In [7]:
print(factura.data.to_json())

{
  "config": {
    "tipo_peaje": "NOC",
    "potencia_contratada": 4.6,
    "con_bono_social": false,
    "zona_impuestos": "IVA",
    "alquiler_anual": 9.72,
    "impuesto_electrico": 0.0511269632,
    "cups": "ES00XXXXXXXXXXXXXXDB"
  },
  "num_dias_factura": 30,
  "start": "2020-02-17 00:00:00",
  "end": "2020-03-18 00:00:00",
  "periodos_fact": [
    {
      "billed_days": 30,
      "year": 2020,
      "termino_fijo_peaje_acceso": 14.34,
      "termino_fijo_comercializacion": 1.17,
      "termino_fijo_total": 15.51,
      "energy_periods": [
        {
          "name": "P1",
          "coste_peaje_acceso_tea": 10.544458468,
          "coste_energia_tcu": 8.477372039999999,
          "energia_total": 170.03900000000002
        },
        {
          "name": "P2",
          "coste_peaje_acceso_tea": 0.67090578,
          "coste_energia_tcu": 13.146024950000003,
          "energia_total": 302.892
        }
      ]
    }
  ],
  "descuento_bono_social": 0.0,
  "termino_impuesto_electric